# Prompt + LLM

这是最基本的使用
Prompt -> PromptTemplate
LLM -> ChatOpenAI

In [9]:
from dotenv import load_dotenv

load_dotenv()

True

## 基础使用

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import ChatOpenAI

# Prompt invoke接受的是Dict
prompt = PromptTemplate.from_template("tell me a joke about {foo}")
# Model invoke接收的可以是string 或者 PromptValue
model = ChatOpenAI()

chain = prompt | model

chain.invoke({"foo": "bears"})

AIMessage(content="Why don't bears wear shoes?\n\nBecause they already have bear feet!")

## 如何给模型传递其他参数

In [6]:
# Bind arguments to a Runnable, returning a new Runnable.
chain_kwargs = prompt | model.bind(stop=["\n"])
chain_kwargs.invoke({"foo": "bears"})

AIMessage(content="Why don't bears wear shoes?")

## 传递Function Call参数

In [11]:
functions = [
    {
        "name": "joke",
        "description": "a joke",
        "parameters": {
            "type": "object",
            "properties": {
                "setup": {"type": "string", "description": "the setup for the joke"},
                "punchline": {
                    "type": "string",
                    "description": "the punchline for the joke"
                }
            },
            "required": ["setup", "punchline"]
        }
    }
]

chain_functions = prompt | model.bind(functions=functions)
chain_functions.invoke({"foo": "bears"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n"setup": "Why don\'t bears like fast food?",\n"punchline": "Because they can\'t catch it!"\n}', 'name': 'joke'}})

## 配合OutputPaarser使用
### 正常String 解析
Str解析的是AIMessage，langchain自己封装的对象

In [13]:
from langchain_core.output_parsers import StrOutputParser

chain_output_parser = chain_kwargs | StrOutputParser()

chain_output_parser.invoke({"foo": "bears"})

"Why don't bears wear shoes?"

### Function返回解析
OpenAI的Functions具有特殊性，所以需要对应OpenAI的解析逻辑

In [15]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain_output_parser = chain_functions | JsonOutputFunctionsParser()

chain_output_parser.invoke({"foo": "bears"})

{'setup': "Why don't bears wear shoes?",
 'punchline': 'Because they have bear feet!'}

In [17]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

chain_output_parser2 = chain_functions | JsonKeyOutputFunctionsParser(key_name="setup")

chain_output_parser2.invoke({"foo": "bears"})

"Why don't bears wear shoes?"

# 简化输入

这里出现了2个需要理解的新对象 `RunnableParallel`
A runnable that runs a mapping of runnables in parallel, and returns a mapping of their outputs.
简单来说就是一个Map_input, 并发执行每一个Item(Step), 然后返回Map_output

`RunnablePassthrough`
A runnable to passthrough inputs unchanged or with additional keys
解释接受Input的值

In [19]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

map_ = RunnableParallel(foo=RunnablePassthrough())

chain_input1 = (
    map_ 
    | prompt
    | model
    | StrOutputParser()
)

chain_input1.invoke("bears")


"Sure, here's a bear joke for you:\n\nWhy don't bears ever wear shoes?\n\nBecause they have bear feet!"

刚才讲了RunnbaleParallel其实就是一个特殊的Map,所有支持这样的Dict的语法糖

In [20]:
chain_input2 = (
    {
        "foo": RunnablePassthrough()
    }
    | prompt
    | model
    | StrOutputParser()
)

chain_input2.invoke("bears")

"Sure, here's a bear joke for you:\n\nWhy don't bears wear shoes?\n\nBecause they have bear feet!"